In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# Hyper-parameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [6]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

In [7]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='/home/hasu/Documents/github/content/CIFAR10/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='/home/hasu/Documents/github/content/CIFAR10/',
                                            train=False, 
                                            transform=transforms.ToTensor())

100%|█████████████████████████| 170498071/170498071 [13:25<00:00, 211765.73it/s]


Extracting /home/hasu/Documents/github/content/CIFAR10/cifar-10-python.tar.gz to /home/hasu/Documents/github/content/CIFAR10/


In [8]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [9]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [10]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [13]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [14]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [19]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/10], Step [100/500] Loss: 1.5176
Epoch [1/10], Step [200/500] Loss: 1.2218
Epoch [1/10], Step [300/500] Loss: 1.2043
Epoch [1/10], Step [400/500] Loss: 1.0456
Epoch [1/10], Step [500/500] Loss: 1.0627
Epoch [2/10], Step [100/500] Loss: 1.0785
Epoch [2/10], Step [200/500] Loss: 1.0949
Epoch [2/10], Step [300/500] Loss: 0.8834
Epoch [2/10], Step [400/500] Loss: 0.8388
Epoch [2/10], Step [500/500] Loss: 0.8072
Epoch [3/10], Step [100/500] Loss: 0.8588
Epoch [3/10], Step [200/500] Loss: 0.8015
Epoch [3/10], Step [300/500] Loss: 0.7583
Epoch [3/10], Step [400/500] Loss: 0.7039
Epoch [3/10], Step [500/500] Loss: 0.6914
Epoch [4/10], Step [100/500] Loss: 0.7519
Epoch [4/10], Step [200/500] Loss: 0.6961
Epoch [4/10], Step [300/500] Loss: 0.6788
Epoch [4/10], Step [400/500] Loss: 0.6671
Epoch [4/10], Step [500/500] Loss: 0.7893
Epoch [5/10], Step [100/500] Loss: 0.7344
Epoch [5/10], Step [200/500] Loss: 0.7259
Epoch [5/10], Step [300/500] Loss: 0.8660
Epoch [5/10], Step [400/500] Loss:

In [20]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 75.36 %
